In [1]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import uniform
import tqdm

In [2]:
# CI function
def CI(data,alpha):
    sample_mean=np.mean(data) # data is a list!
    sample_sigma=np.std(data)
    critical_point = norm.ppf(1-alpha/2)
    LB=sample_mean-critical_point*sample_sigma/np.sqrt(len(data))
    UB=sample_mean+critical_point*sample_sigma/np.sqrt(len(data))
    return LB,UB

# Question 1:

## (a)

In [3]:
# parameters
r,div,sigma,rho=0.05,0.02,0.3,0.2
S10,S20,S30,K,T=100,100,100,100,1
N,size = 252,100000 # N:steps size:simulation times
h=T/N
# simulate standard normal
Z1=np.random.normal(0,1,(N,size))
Z2=np.random.normal(0,1,(N,size))
Z3=np.random.normal(0,1,(N,size))
X1=Z1
X2=rho*Z1+np.sqrt(1-(rho**2))*Z2
X3=rho*Z1+((rho-rho**2)/np.sqrt(1-rho**2))*Z2+(np.sqrt(1-rho**2-((rho-rho**2)**2)/(1-rho**2)))*Z3
# simulate stock price: St=S0*exp{(miu-0.5*sigma^2)*t+sigma*Wt}
S1=np.ones((1,size))*S10
S2=np.ones((1,size))*S20
S3=np.ones((1,size))*S30
zero = np.zeros(size)
for i in tqdm.tqdm(range(N)):
    a=S1[i,:]*np.exp((r-div-(sigma**2)*0.5)*(h)+sigma*np.sqrt(h)*X1[i,:])
    b=S2[i,:]*np.exp((r-div-(sigma**2)*0.5)*(h)+sigma*np.sqrt(h)*X2[i,:])
    c=S3[i,:]*np.exp((r-div-(sigma**2)*0.5)*(h)+sigma*np.sqrt(h)*X3[i,:])
    S1=np.append(S1,[a],axis=0)
    S2=np.append(S2,[b],axis=0)
    S3=np.append(S3,[c],axis=0)
payoff=np.max([S1[-1,:]+S2[-1,:]-S3[-1,:]-K,zero],axis=0) 
# Discount to now

100%|██████████| 252/252 [00:27<00:00,  9.07it/s]


In [4]:
payoff

array([ 0.        ,  0.        ,  0.        , ..., 27.85993772,
       39.0736305 , 66.45905357])

In [6]:
CI(payoff,0.05)

(21.32475606149343, 21.736769029358165)

In [7]:
np.mean(payoff)

21.530762545425798

In [9]:
np.var(payoff/N)

0.01739660552946509

## (b)

In [18]:
# part b

r=0.05
yie=0.02
sigma=0.3
rho=0.2
S1=100
S2=100
S3=100
K=100
T=1
N=100000 # simulation time
m=12 # steps
dt=T/m
M=10 # number of polynomial


S1=S1*np.ones((m+1,N))
S2=S2*np.ones((m+1,N))
S3=S3*np.ones((m+1,N))

Normal_1=np.random.normal(0,1,(m,N))
Normal_2=np.random.normal(0,1,(m,N))
Normal_3=np.random.normal(0,1,(m,N))

Z_1=Normal_1
Z_2=rho*Normal_1+np.sqrt(1-rho**2)*Normal_2
Z_3=rho*Normal_1+((rho-rho**2)/np.sqrt(1-rho**2))*Normal_2+np.sqrt(1-rho**2-(rho-rho**2)**2/((1-rho**2)))*Normal_3


for i in range(0,m):
    S1[i+1,:]=S1[i,:]*np.exp((r-yie-sigma**2/2)*dt+sigma*np.sqrt(dt)*Z_1[i,:])
    S2[i+1,:]=S2[i,:]*np.exp((r-yie-sigma**2/2)*dt+sigma*np.sqrt(dt)*Z_2[i,:])
    S3[i+1,:]=S3[i,:]*np.exp((r-yie-sigma**2/2)*dt+sigma*np.sqrt(dt)*Z_3[i,:])

# Generate terminal payoff
V=np.zeros((m+1,N))
V[m,:]=np.where(S1[m]+S2[m]-S3[m]-K>0,S1[m]+S2[m]-S3[m]-K,0)

# Backward calculate "Continue value" for each time point
for i in tqdm.tqdm(range(m-1,0,-1)):
    psi=np.zeros((M,N))
    
    psi[0,:]=1
    psi[1,:]=S1[i,:]
    psi[2,:]=S2[i,:]
    psi[3,:]=S3[i,:]
    psi[4,:]=S1[i,:]**2
    psi[5,:]=S2[i,:]**2
    psi[6,:]=S3[i,:]**2
    psi[7,:]=S1[i,:]**3
    psi[8,:]=S2[i,:]**3
    psi[9,:]=S3[i,:]**3
    
    # Continue Value
    Bpsi=np.dot(psi,np.transpose(psi))
    beta=np.dot(np.dot(np.linalg.inv(Bpsi),psi),np.transpose(V[i+1,:]))
    CV=np.exp((yie-r)*dt)*np.dot(np.transpose(beta),psi)
    
    # Exercise immediately payoff 
    v=np.where(S1[i+1]+S2[i+1]-S3[i+1]-K>0,S1[i+1]+S2[i+1]-S3[i+1]-K,0)
    
    # Compare with each other
    V[i,:]=np.where(CV>v,CV,v)
Value=np.max(np.exp((yie-r)*dt)*np.mean(V[1]),0)

print(Value,CI(np.exp((yie-r)*dt)*V[1],0.05),np.var(V[1])/N)

100%|██████████| 11/11 [00:00<00:00, 48.59it/s]

35.70219003305834 (35.642500792034, 35.76187927408266) 0.0009321105075052054


# Question 2:

## (a)

In [36]:
S1=50
K1=51
sigma1=0.28
T1=0.75

S2=20
K2=19
sigma2=0.25
T2=1
r=0.06
rho=0.4
N=1000000

delta_c=norm.cdf((np.log(S1/K1)+(r+sigma1**2/2)*T1)/(sigma1*np.sqrt(T1)))
delta_p=norm.cdf((np.log(S2/K2)+(r+sigma2**2/2)*T2)/(sigma2*np.sqrt(T2)))-1

Z_1=np.random.normal(0,1,N)
Z_2=rho*Z_1+np.sqrt(1-rho**2)*np.random.normal(0,1,N)

dx1=Z_1*sigma1/np.sqrt(252) # volatilty is annual 
dx2=Z_2*sigma2/np.sqrt(252)
dV1=delta_c*S1*dx1
dV2=delta_p*S2*dx2
dV=dV1+dV2

VaR=-np.percentile(dV,1)*np.sqrt(10)
print(VaR)

3.60798123303267


## (b)

In [37]:
# part b
S1=50
K1=51
sigma1=0.28
T1=0.75

S2=20
K2=19
sigma2=0.25
T2=1
r=0.06
rho=0.4
N=1000000

delta_c=norm.cdf((np.log(S1/K1)+(r+sigma1**2/2)*T1)/(sigma1*np.sqrt(T1)))
delta_p=norm.cdf((np.log(S2/K2)+(r+sigma2**2/2)*T2)/(sigma2*np.sqrt(T2)))-1

Z_1=np.random.normal(0,1,N)
Z_2=rho*Z_1+np.sqrt(1-rho**2)*np.random.normal(0,1,N)
dx1=Z_1*sigma1/np.sqrt(252)
dx2=Z_2*sigma2/np.sqrt(252)

# second order of C/S **remind chain rule
gamma1=norm.pdf((np.log(S1/K1)+(r+sigma1**2/2)*T1)/(sigma1*np.sqrt(T1)))/(S1*sigma1*np.sqrt(T1)) 
gamma2=norm.pdf((np.log(S2/K2)+(r+sigma2**2/2)*T2)/(sigma2*np.sqrt(T2)))/(S2*sigma2*np.sqrt(T2))

dV1=delta_c*S1*dx1+0.5*gamma1*(S1**2)*(dx1**2)
dV2=delta_p*S2*dx2+0.5*gamma2*(S2**2)*(dx2**2)
dV=dV1+dV2

VaR=-np.percentile(dV,1)*np.sqrt(10)
print(VaR)

3.389627728545039


## (c)

In [38]:
# part c
N=1000000
S1=50
K1=51
sigma1=0.28
T1=0.75

S2=20
K2=19
sigma2=0.25
T2=1
r=0.06
rho=0.4


Z_1=np.random.normal(0,1,N)
Z_2=rho*Z_1+np.sqrt(1-rho**2)*np.random.normal(0,1,N)

def C(S,T):
    return S*norm.cdf((np.log(S/K1)+(r+sigma1**2/2)*T)/(sigma1*np.sqrt(T)))-K1*np.exp(-r*T)*norm.cdf((np.log(S/K1)+(r-sigma1**2/2)*T)/(sigma1*np.sqrt(T)))
def P(S,T):
    return -S*norm.cdf((np.log(K2/S)-(r+sigma2**2/2)*T)/(sigma2*np.sqrt(T)))+K2*np.exp(-r*T)*norm.cdf((np.log(K2/S)-(r-sigma2**2/2)*T)/(sigma2*np.sqrt(T)))

h=1/252
S1_h=S1*np.exp((r-sigma1**2/2)*h+sigma1*np.sqrt(h)*Z_1)
S2_h=S2*np.exp((r-sigma2**2/2)*h+sigma2*np.sqrt(h)*Z_2)

# t=0 portfolio value
p_0=C(S1,T1)+P(S2,T2)
# t=1 portfolio value **remind: maturity date would minus 1.
p_1=C(S1_h,T1-h)+P(S2_h,T2-h)
dV=p_0-p_1
VaR=np.percentile(dV,1)*(-np.sqrt(10))
print(VaR)

3.8475259215572173


# Question 3:

In [20]:
V0=100
u=0.03
sigma=0.4
B=70
T=5

m=252 # steps
h=T/m
N=100000

V=np.ones((m+1,N))*V0
M=np.zeros((m,N))

Z=np.random.normal(0,1,(m,N))
U=uniform.rvs(size=(m,N))

num=[]
for n in range(N):
    for i in range(m):
        V[i+1,n]=V[i,n]*np.exp((u-sigma**2/2)*h+sigma*np.sqrt(h)*Z[i,n])
        M[i,n]=np.exp(0.5*(np.log(V[i+1,n]*V[i,n])-np.sqrt((np.log(V[i+1,n]/V[i,n]))**2-2*sigma**2*h*np.log(U[i,n]))))
        if M[i,n]<B:
            num.append(1)
            break
        if i==m-1:
            num.append(0)
print(np.mean(num),CI(num,0.05))

0.76447 (0.7618400261480249, 0.767099973851975)
